
# COMP305 -> 2-median problem on Optimal Placement of 2 Hospitals

## Imports

In [1]:
import time

import heapq
import numpy as np
from collections import defaultdict
from collections import Counter

from random import choice
from random import randint

## Data Read

In [2]:
with open("tests/test1_new.txt") as f:
    test1 = f.read().splitlines()   

with open("tests/test2_new.txt") as f:
    test2 = f.read().splitlines()   
    


## Change the test to be run below

In [3]:
lines = test2

# txt -> Graph

In [4]:
number_of_vertices = int(lines[0])
number_of_edges = int(lines[1])

vertices = lines[2:2+number_of_vertices]
edges = lines[2+number_of_vertices:]

ids_and_populations = [tuple(map(int, vertices[i].split(" "))) for i in range(len(vertices))]
populations = dict(sorted(dict(ids_and_populations).items())) #redundant sort

mydict = lambda: defaultdict(lambda: defaultdict())
G = mydict()

for i in range(len(edges)):
    source, target, weight = map(int, edges[i].split(" "))
    G[source][target] = weight
    G[target][source] = weight
    

# Our Binary Heap and Priority Queue Implementation

In [5]:
class PriorityQueue:
    """Priority Queue implemented with binary heap for efficiency"""

    def __init__(self):
        """Constructor for the binary heap"""
        self.size = 0
        self.capacity = 0
        self.array = []

    def is_empty(self):
        """Returns true if queue is empty false otherwise"""
        return self.size == 0

    def left_child(self, index):
        """Returns the left child of the node given as an index"""
        return (2 * index) + 1

    def right_child(self, index):
        """Returns the right child of the node given as an index"""
        return (2 * index) + 2

    def parent(self, index):
        """Returns the parent of the node given as an index"""
        return (index - 1) // 2

    def hasleftchild(self, index):
        """Checks if the indexed element has a left child"""
        return self.left_child(index) < len(self.array)

    def hasrightchild(self, index):
        """Checks if the indexed element has a right child"""
        return self.right_child(index) < len(self.array)

    def pop(self):
        """Returns the smallest element from the list and removes the element. Asserts a error if it is called on a
        empty list """
        if self.is_empty():
            assert "Empty List"

        min_el = self.array[0]
        self.array[0] = self.array[len(self.array) - 1]
        del self.array[len(self.array) - 1]
        self.downheap(0)
        return min_el

    def view(self):
        """Returns the smallest element from the list without removing it. Asserts a error if it is called on a
        empty list """
        if self.is_empty():
            assert "Empty List"

        return self.array[0]

    def push(self, element):
        """Adds the given element to the queue"""
        self.array.append(element)
        self.upheap(len(self.array) - 1)

    def upheap(self, index):
        """Corrects the heap property top down if it is broken"""
        while index > 0:
            parent = self.parent(index)
            if self.array[parent] == self.array[index] or self.array[index] > self.array[parent]:
                break
            temp = self.array[parent]
            self.array[parent] = self.array[index]
            self.array[index] = temp
            index = parent

    def __str__(self):
        """Prints the queue"""
        string = "[ "
        for k in range(0, len(self.array)):
            if k % 20 == 0 and k != 0:
                string += "\n"
            string += self.array[k].__str__() + " "
        string += "]\n"
        return string

    def downheap(self, index):
        """Corrects the heap property  bottom up if it is broken"""
        while self.hasrightchild(index) or self.hasleftchild(index):
            small = self.left_child(index)

            if self.hasrightchild(index):
                rc = self.right_child(index)

                if self.array[rc] < self.array[small]:
                    small = rc

            if self.array[small] == self.array[index] or self.array[small] > self.array[index]:
                break
            temp = self.array[small]
            self.array[small] = self.array[index]
            self.array[index] = temp
            index = small


if __name__ == '__main__':
    q = PriorityQueue()


## Heuristic Algorithms' Utilities

In [6]:
def select_neighbors(G, sub_graph, current_node, k):
    if k == 0:
        return sub_graph

    for j in G[current_node].items():
        sub_graph[current_node][j[0]] = j[1]
        sub_graph[j[0]][current_node] = j[1]

        sub_graph = select_neighbors(G, sub_graph, j[0], k - 1)
    return sub_graph


def merge_graph(dict1, dict2):
    for key, value in dict2.items():
        for subkey, subvalue in value.items():
            dict1[key][subkey] = subvalue

def dijkstra(G, populations, source): 
    costs = dict()
    for key in G:
        costs[key] = np.inf
    costs[source] = 0

    pq = PriorityQueue()
    for node in G:
        pq.push((node, costs[node]))

    while len(pq.array) != 0:
        current_node, current_node_distance = pq.pop()
        for neighbor_node in G[current_node]:
            weight = G[current_node][neighbor_node]
            distance = current_node_distance + weight
            if distance < costs[neighbor_node]:
                costs[neighbor_node] = distance
                pq.push((neighbor_node, distance))
    
    sorted_costs_lst=list(dict(sorted(costs.items())).values())
    populations_values_lst = list(dict(sorted(populations.items())).values())
    return np.sum(np.array(sorted_costs_lst) * np.array(populations_values_lst))

def dijkstra_q_impl(G, populations, source): 
    costs = dict()
    for key in G:
        costs[key] = np.inf
    costs[source] = 0

    pq = []
    for node in G:
         pq.append((node, costs[node]))

    while len(pq) != 0:
        current_node, current_node_distance = pq.pop(0)
        for neighbor_node in G[current_node]:
            weight = G[current_node][neighbor_node]
            distance = current_node_distance + weight
            if distance < costs[neighbor_node]:
                costs[neighbor_node] = distance
                pq.append((neighbor_node, distance))
    
    sorted_costs_lst=list(dict(sorted(costs.items())).values())
    populations_values_lst = list(dict(sorted(populations.items())).values())
    return np.sum(np.array(sorted_costs_lst) * np.array(populations_values_lst))

def random_start(G):
    res = [choice(list(G.keys())), choice(list(G.keys()))]
    if res[0] == res [1]:
        return random_start(G)
    print(f"Random start: {res}")
    return res


#//2 * O((V+E)*logV) = O(E*logV) // 
def allocation_cost(G, population_dict, i,j):
    return [dijkstra(G,population_dict, i),dijkstra(G,population_dict, j)]


# V times Dijkstra
def sub_graph_apsp(G, dijkstra_func):
    population_dict = dict(sorted([(k, populations[k]) for k in G.keys()]))
    selected_vertex = choice(list(G.keys()))
    selected_cost = dijkstra_func(G,population_dict, selected_vertex)
    
    for node in G.keys():
        if node is not selected_vertex:
            this_cost = dijkstra_func(G, population_dict, node) 
            if this_cost < selected_cost:
                selected_cost = this_cost
                selected_vertex = node    
    return selected_vertex, selected_cost


def algorithm_sub_graph_apsp(G, starting_node, k, hop_list, dijkstra_func):
    sub_graph = lambda: defaultdict(lambda: defaultdict())
    sub_graph = sub_graph()
    sub_graph = select_neighbors(G, sub_graph, current_node=starting_node, k=k)
    next_node, cost = sub_graph_apsp(sub_graph, dijkstra_func)
    
    if len(hop_list) > 0 and next_node == hop_list[-1][0]:
        return next_node, cost
    
    hop_list.append((next_node, cost))
    return algorithm_sub_graph_apsp(G, next_node, k, hop_list, dijkstra_func)


def regional_interchange(G,current_node,k):
    #k-th neighbor subgraph
    sub_graph = lambda: defaultdict(lambda: defaultdict())
    sub_graph = sub_graph()
    select_neighbors(G, sub_graph, current_node, k)

    return sub_graph_apsp(sub_graph,dijkstra_func=dijkstra_q_impl)[0]



from multiprocessing import Process

res_queue = [] # global queue for threads to enqueue

        
def Teitz_Bart_Spawn_helper(G, k):
    global res_queue
    
    population_dict = dict(sorted([(k, populations[k]) for k in G.keys()]))
    selected_vertices = random_start(G) 
    selected_costs = allocation_cost(G,population_dict, selected_vertices[0],selected_vertices[1])

    counter=0
    for not_selected in G.keys():
        counter+=1
        k-=1;
        #print(k)
        if k==-1:
            res_queue.append((selected_vertices,selected_costs))
            break
        if not_selected not in selected_vertices:
            bigger = max(selected_costs)
            this_cost = dijkstra(G,population_dict, not_selected) 
            if this_cost < bigger:
                bigger_index = selected_costs.index(bigger)
                selected_costs[bigger_index] = this_cost
                selected_vertices[bigger_index] = not_selected
                res_queue.append((selected_vertices,selected_costs))#is this necessary

    return min(res_queue, key=lambda x:x[1])
    


# Greedy Heuristic Algorithms

In [7]:
# 2*O(V)*O(E*logV) = O(E*V*logV) #
def Greedy_Heuristic_Add_Drop(G, dijkstra_func):
    population_dict = dict(sorted([(k, populations[k]) for k in G.keys()]))
    selected_vertices = random_start(G) 
    selected_costs = allocation_cost(G,population_dict, selected_vertices[0],selected_vertices[1])
    
    for not_selected in G.keys():
        if not_selected not in selected_vertices:
            bigger = max(selected_costs)
            this_cost = dijkstra_func(G,population_dict, not_selected) 
            if this_cost < bigger:
                bigger_index = selected_costs.index(bigger)
                selected_costs[bigger_index] = this_cost
                selected_vertices[bigger_index] = not_selected
    return (selected_vertices,selected_costs)


# n: how many random bootstrap positions to spawn
# k: how many iterations to run per spawned per Greedy_Heuristic_Add_Drop
def Greedy_Heuristic_Add_Drop_Spawn(G, n, k):
    global res_queue
    for i in range(n):
        print('spawning thread '+str(i))
        Process(target=Teitz_Bart_Spawn_helper(G, k)).start()
    return min(res_queue, key=lambda x:x[1])


def Greedy_Heuristic_Subgraph_Expansion(G, k, dijkstra_func, bootstrap_cnt=10):
    nodes = []
    costs = []
    
    for i in range(bootstrap_cnt):
        node, cost = algorithm_sub_graph_apsp(G, choice(list(G.keys())), k, [], dijkstra_func=dijkstra_func)
        nodes.append(node)
        costs.append(cost)
        
    counter = Counter(nodes)
    most_commons = counter.most_common(2)
    target_nodes = (most_commons[0][0], most_commons[1][0])
    
    sub_graph1 = lambda: defaultdict(lambda: defaultdict())
    sub_graph1 = sub_graph1()
    sub_graph1 = select_neighbors(G, sub_graph1, target_nodes[0], k=k)
    
    sub_graph2 = lambda: defaultdict(lambda: defaultdict())
    sub_graph2 = sub_graph2()
    sub_graph2 = select_neighbors(G, sub_graph2, target_nodes[1], k=k)

    merge_graph(sub_graph1, sub_graph2)

    points, costs = Greedy_Heuristic_Add_Drop(sub_graph1, dijkstra_func)

    if np.inf in costs:
        print("INF")
        sub_graph1 = lambda: defaultdict(lambda: defaultdict())
        sub_graph1 = sub_graph1()
        sub_graph1 = select_neighbors(G, sub_graph1, current_node=points[0], k=k+1)
        
        sub_graph2 = lambda: defaultdict(lambda: defaultdict())
        sub_graph2 = sub_graph2()
        sub_graph2 = select_neighbors(G, sub_graph2, current_node=points[1], k=k+1)
        
        merge_graph(sub_graph1, sub_graph2)
        
        

        points, costs = Greedy_Heuristic_Add_Drop(sub_graph1, dijkstra_func)


            
            
        if np.inf not in costs:
            return points, costs
        else:
            print("Graphs are disconnected. Total cost is inf")
            return points, costs
    return points, costs

# Global/Regional Interchange Algorithm
def Greedy_Heuristic_Local_Interchange(G,k):
    population_dict = dict(sorted([(k, populations[k]) for k in G.keys()]))
    selected_vertices = random_start(G) 
    selected_costs = allocation_cost(G,population_dict, selected_vertices[0],selected_vertices[1])
    
    for not_selected in G.keys():
        if not_selected not in selected_vertices:
            bigger = max(selected_costs)
            this_cost = dijkstra(G,population_dict, not_selected) 
            if this_cost < bigger:
                bigger_index = selected_costs.index(bigger)
                selected_costs[bigger_index] = this_cost
                
                temp = regional_interchange(G,not_selected,k)
                if temp != min(selected_vertices):
                    selected_vertices[bigger_index] = temp
                # Regional Interchange
                # k-th neighbor -> 5. order neighbor subgraph
                    # V^2*logV 1-hospital problem
    return(selected_vertices,selected_costs)



# Greedy Heuristic Runs

In [8]:
start = time.time()

res = Greedy_Heuristic_Add_Drop(G,dijkstra_func=dijkstra )

diff = time.time()-start

print('\npick cities #'+  str(res[0]) +' with costs '+ str(res[1]))
print('\ntotal time: '+ str(diff)+ ' sec')


Random start: [212, 245]

pick cities #[71, 157] with costs [13478689, 13484179]

total time: 22.702250957489014 sec


In [9]:

start = time.time()

res = Greedy_Heuristic_Subgraph_Expansion(G, 5, bootstrap_cnt=10, dijkstra_func=dijkstra) #q for direct Queue based PQ impl (py's pop(0))

diff = time.time()-start

print('\npick cities #'+  str(res[0]) +' with costs '+ str(res[1]))
print('\ntotal time using our Binary-Heap PQ: '+ str(diff)+ ' sec')

Random start: [141, 149]

pick cities #[157, 71] with costs [9950365, 9944943]

total time using our Binary-Heap PQ: 110.01797890663147 sec


In [10]:
start = time.time()

res = Greedy_Heuristic_Subgraph_Expansion(G, 5, bootstrap_cnt=10, dijkstra_func=dijkstra_q_impl) #q for direct Queue based PQ impl (py's pop(0))

diff = time.time()-start

print('\npick cities #'+  str(res[0]) +' with costs '+ str(res[1]))
print('\ntotal time using our direct-Queue-based PQ: '+ str(diff)+ ' sec')

Random start: [82, 9]

pick cities #[71, 157] with costs [7834750, 7837472]

total time using our direct-Queue-based PQ: 12.161312103271484 sec


In [11]:
start = time.time()

res = Greedy_Heuristic_Local_Interchange(G, 5)

diff = time.time()-start

print('\npick cities #'+  str(res[0]) +' with costs '+ str(res[1]))
print('\ntotal time: '+ str(diff)+ ' sec')

Random start: [144, 25]

pick cities #[214, 71] with costs [13484179, 13478689]

total time: 25.314204931259155 sec


# Dynamic Programming + Brute Force Combination Selection Algorithm

### A Dijkstra utility specifically for paths and our V^4 Algorithm

In [12]:
def dijkstra_path(G, population_dict, source):
    costs = dict()
    for key in G:
        costs[key] = np.inf
    costs[source] = 0

    pq = []
    for node in G:
        heapq.heappush(pq, (node, costs[node]))

    while len(pq) != 0:
        current_node, current_node_distance = heapq.heappop(pq)
        for neighbor_node in G[current_node]:
            weight = G[current_node][neighbor_node]
            distance = current_node_distance + weight
            if distance < costs[neighbor_node]:
                costs[neighbor_node] = distance
                heapq.heappush(pq, (neighbor_node, distance))
   
    sorted_costs_lst=list(dict(sorted(costs.items())).values())
    sorted_populations_lst = list(dict(sorted(population_dict.items())).values())
    return np.array(sorted_costs_lst) * np.array(sorted_populations_lst)
    #return list(dict(sorted(costs.items())).values())
   
   
# V4 because runs in V^4
def V4(G):

    APSP = np.zeros((number_of_vertices,number_of_vertices))
    population_dict = dict(sorted([(k, populations[k]) for k in G.keys()]))
    for vertex in vertices:
        vertex= int(vertex.split()[0])
        APSP[vertex] = [e for e in dijkstra_path(G, population_dict,vertex)]

    global glob

    res = {}

    n = len(APSP)
   
    temp_arr = APSP.copy()

    count=0
    count2=0
    for first in range(n):
        for second in range(first+1,n):
            if first==second:
                continue
            count+=1
            temp_arr = APSP.copy()
            for row in temp_arr:
                if row[first]<row[second]:
                    row[second]=0
                else:
                    row[first]=0
            to_be_summed = temp_arr[:,[first,second]]
            summed = sum(sum(to_be_summed))
            res[(first,second)]=summed
    ret=min(res, key=res.get)
    return ret, res[ret], res

In [13]:
start = time.time()

res = V4(G)

diff = time.time()-start

print('\npick cities #'+  str(res[0]) +' with costs '+ str(res[1]))
print('\ntotal time: '+ str(diff)+ ' sec')


pick cities #(205, 215) with costs 121635.0

total time: 21.395344257354736 sec


# Cutting The Graph and Running Floyd-Warshall s

### Floyd-Warshall Utilities

In [14]:
import readfile

def floydwarshall_helper(graph, root, root_index, weight):

    dist = [ [0]*len(graph) for i in range(len(graph))]
    marked = [ [0]*len(graph) for i in range(len(graph))]

    for x in range(len(graph)):
        if graph[x][root_index] == float('inf'):
            continue
        graph[x][root_index] = 0

    for x in range(len(graph)):
        if graph[root_index][x] == float('inf'):
            continue
        graph[root_index][x] = 0

    for x in range(len(graph)):
        for y in range(len(graph)):
            dist[x][y] = graph[x][y]

    for x in range(len(graph)):
        for y in range(len(graph)):
            marked[x][y] = 0

    for z in range(len(graph)):
        for x in range(len(graph)):
            for y in range(len(graph)):

                if dist[x][z] == float('inf') or dist[z][y] == float('inf'):
                    continue

                if dist[x][z] + dist[z][y] < dist[x][y]:
                    dist[x][y] = dist[x][z] + dist[z][y]
                    if z == root_index: 

                        marked[x][y] = 1
                        marked[y][x] = 1
                        marked[x][z] = 1
                        marked[z][x] = 1
                        marked[z][z] = 1

                    if marked[x][z] == 1 or marked[z][y] == 1:
                        marked[x][y] = 1

    value = [0 for i in range(len(graph))]
    value[root_index] = float('inf')

    for x in range(len(graph)):
        for y in range(len(graph)):
            if dist[x][y] == float('inf'):
                continue
            if value[x] == float('inf'):
                continue

            if marked[x][y] == 1:
                continue
            value[x] += dist[x][y] * weight[x][1]

    min1 = float('inf')
    min2 = float('inf')
    min2_index = 0
    min1_index = 0

    for x in range(len(graph)):
        if value[x] <= min2:

            if value[x] <= min1:
                min2 = min1
                min2_index = min1_index
                min1 = value[x]
                min1_index = x
            else:
                min2 = value[x]
                min2_index = x

    print("Selected min 1 is : ")
    print(min1_index)
    print("\n")
    print("Selected min 2 is : ")
    print(min2_index)
    print("\n")


def floydwarshall(graph, weight):
    dist = [ [0]*len(graph) for i in range(len(graph))]
    marked = [ [0]*len(graph) for i in range(len(graph))]
    root = float('inf')
    root_index = 0

    for x in range(len(graph)):
        for y in range(len(graph)):
            dist[x][y] = graph[x][y]
            marked[x][y] = 0

    for z in range(len(graph)):
        for x in range(len(graph)):
            for y in range(len(graph)):

                if dist[x][z] == float('inf') or dist[z][y] == float('inf'):
                    continue

                if dist[x][z] + dist[z][y] < dist[x][y]:
                    dist[x][y] = dist[x][z] + dist[z][y]

    value = [0 for i in range(len(graph))]
    root = float('inf')

    for x in range(len(graph)):
        for y in range(len(graph)):
            if dist[x][y] == float('inf'):
                continue
            value[x] += dist[x][y] * weight[y][1]

    for x in range(len(graph)):
        if value[x] <= root:
            root = value[x]
            root_index = x

    print("Root value for the graph is ")
    print(root_index)
    print("\n")
    floydwarshall_helper(graph, root, root_index, weight)

    
    
if __name__ == '__main__':
    graph, weight = readfile.readfile("tests/test2_new.txt")
    start = time.time()

    floydwarshall(graph, weight)
    
    diff = time.time()-start

    print('\ntotal time: '+ str(diff)+ ' sec')
    

Root value for the graph is 
71


Selected min 1 is : 
233


Selected min 2 is : 
230



total time: 14.544553756713867 sec
